<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_p3_UNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cpu


In [3]:
import os


train_paths = '/content/drive/MyDrive/NTU_DLCV/p3_data/train'
img_paths_train = sorted([os.path.join(train_paths, i) for i in os.listdir(train_paths) if i.endswith('.jpg')])
mask_paths_train = sorted([os.path.join(train_paths, i) for i in os.listdir(train_paths) if i.endswith('.png')])

val_paths = '/content/drive/MyDrive/NTU_DLCV/p3_data/validation'
img_paths_val = sorted([os.path.join(val_paths, i) for i in os.listdir(val_paths) if i.endswith('.jpg')])
mask_paths_val = sorted([os.path.join(val_paths, i) for i in os.listdir(val_paths) if i.endswith('.png')])

In [4]:
len(img_paths_train), len(mask_paths_train), len(img_paths_val), len(mask_paths_val)

(2000, 2000, 257, 257)

In [5]:
import torch
import random
import numpy as np
from PIL import Image
from copy import deepcopy
from torchvision.transforms.functional import hflip, vflip, to_tensor, normalize


class LandDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, mask_paths, transform, mode, augment=False):
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        self.transform = transform
        self.mode = mode

        # Data Augmentation
        def original(x): return x
        if augment:
            self.augment = [original, hflip, vflip, original]
        else:
            self.augment = [original]

        # double check the len of img and mask.
        assert len(self.img_paths) == len(self.mask_paths)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        # get img path
        img_path = self.img_paths[idx]
        # Read img
        img = Image.open(img_path).convert('RGB')
        # Normalize Image with VGG16's mean and std
        # (H, W, C) -> (C, H, W)
        img = self.transform(img)

        if self.mode != 'test':
            # get mask path
            mask_path = self.mask_paths[idx]
            mask = Image.open(mask_path).convert('RGB')
            mask = np.array(mask)
            # Binarize mask from [0~255] to (0 or 1)
            mask = (mask >= 128).astype(int)

            # squeeze [a,b,c] into [x,y] with 7 category(6 classes + 1 background)
            mask = 100 * mask[:, :, 0] + 10 * mask[:, :, 1] + 1 * mask[:, :, 2]
            raw_mask = deepcopy(mask) # Only perform numerical conversion on the most original data
            mask[raw_mask == 11] = 0  # (Cyan: 011) Urban land
            mask[raw_mask == 110] = 1  # (Yellow: 110) Agriculture land
            mask[raw_mask == 101] = 2  # (Purple: 101) Rangeland
            mask[raw_mask == 10] = 3  # (Green: 010) Forest land
            mask[raw_mask == 1] = 4  # (Blue: 001) Water
            mask[raw_mask == 111] = 5  # (White: 111) Barren land
            mask[raw_mask == 0] = 6  # (Black: 000) Unknown
            mask = torch.tensor(mask)

            # random Data Augmentation
            augmentor = random.choice(self.augment)
            img = augmentor(img)
            mask = augmentor(mask)

            # mask = mask.to(torch.float)  # 将目标标签转换为浮点数类型
            return img, mask
        else:
            return img

In [6]:
import torchvision.transforms as tr


# IMG_SIZE : 512*512
BATCH_SIZE = 10

# VGG16_V1, https://pytorch.org/vision/main/models/generated/torchvision.models.vgg16.html
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

train_ds = LandDataset(img_paths_train,
                       mask_paths_train,
                       transform = tr.Compose([
                          tr.ToTensor(),
                          tr.Normalize(mean=mean, std=std),
                          ]),
                       mode = 'train',
                       augment = True,)
val_ds = LandDataset(img_paths_val,
                     mask_paths_val,
                     transform = tr.Compose([
                          tr.ToTensor(),
                          tr.Normalize(mean=mean, std=std),
                          ]),
                     mode = 'val',
                     augment = False,)

# num_workers > 0: accelerate loading data by muli-process
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_ds, 1, shuffle=False, num_workers=4)

In [4]:
from torch import nn


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding='same'):
        super(ConvBlock, self).__init__()
        self.convblock = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding='same'),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding='same'),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding='same'),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def __call__(self, x):
        return self.convblock(x)

class UNet(nn.Module):
    def __init__(self, num_classes=7, in_channels=3, init_channels=8):
        super(UNet, self).__init__()
        # Encoder
        self.encoder1 = ConvBlock(in_channels, init_channels) # (3, H, W) -> (64, H, W)
        self.pooling1 = nn.MaxPool2d(2)
        self.encoder2 = ConvBlock(init_channels, init_channels*2) # (64, H/2, W/2) -> (128, H/2, W/2)
        self.pooling2 = nn.MaxPool2d(2)
        self.encoder3 = ConvBlock(init_channels*2, init_channels*4) # (128, H/4, W/4) -> (256, H/4, W/4)
        self.pooling3 = nn.MaxPool2d(2)
        self.encoder4 = ConvBlock(init_channels*4, init_channels*8) # (256, H/8, W/8) -> (512, H/8, W/8)
        self.pooling4 = nn.MaxPool2d(2)
        self.encoder5 = ConvBlock(init_channels*8, init_channels*16) # (256, H/8, W/8) -> (512, H/8, W/8)
        self.pooling5 = nn.MaxPool2d(2)
        self.encoder6 = ConvBlock(init_channels*16, init_channels*32) # (512, H/16, W/16) -> (1024, H/16, W/16)

        # Decoder
        self.upconv5 = nn.ConvTranspose2d(init_channels*32, init_channels*16, kernel_size=2, stride=2) # (1024, H/16, W/16) -> (512, H/8, W/8)
        self.decoder5 = ConvBlock(init_channels*32, init_channels*16)
        self.upconv4 = nn.ConvTranspose2d(init_channels*16, init_channels*8, kernel_size=2, stride=2) # (1024, H/16, W/16) -> (512, H/8, W/8)
        self.decoder4 = ConvBlock(init_channels*16, init_channels*8)
        self.upconv3 = nn.ConvTranspose2d(init_channels*8, init_channels*4, kernel_size=2, stride=2) # (512, H/8, W/8) -> (256, H/4, W/4)
        self.decoder3 = ConvBlock(init_channels*8, init_channels*4)
        self.upconv2 = nn.ConvTranspose2d(init_channels*4, init_channels*2, kernel_size=2, stride=2) # (256, H/4, W/4) -> (128, H/2, W/2)
        self.decoder2 = ConvBlock(init_channels*4, init_channels*2)
        self.upconv1 = nn.ConvTranspose2d(init_channels*2, init_channels, kernel_size=2, stride=2) # (128, H/2, W/2) -> (64, H, W)
        self.decoder1 = ConvBlock(init_channels*2, init_channels)

        # Output
        self.output = nn.Conv2d(init_channels, out_channels=num_classes, kernel_size=1) # (64, H, W) -> (num_classes, H, W)

    def __call__(self, x):
        # Encoder
        encode1 = self.encoder1(x)
        encode2 = self.encoder2(self.pooling1(encode1))
        encode3 = self.encoder3(self.pooling2(encode2))
        encode4 = self.encoder4(self.pooling3(encode3))
        encode5 = self.encoder5(self.pooling4(encode4))
        bottleneck = self.encoder6(self.pooling5(encode5))

        # Decoder
        x = torch.cat((self.upconv5(bottleneck), encode5), dim=1) # (1024, H/8, W/8)
        x = self.decoder5(x) # (512, H/8, W/8)
        x = torch.cat((self.upconv4(x), encode4), dim=1) # (1024, H/8, W/8)
        x = self.decoder4(x) # (512, H/8, W/8)
        x = torch.cat((self.upconv3(x), encode3), dim=1) # (512, H/4, W/4)
        x = self.decoder3(x) # (256, H/4, W/4)
        x = torch.cat((self.upconv2(x), encode2), dim=1) # (512, H/2, W/2)
        x = self.decoder2(x) # (128, H/4, W/4)
        x = torch.cat((self.upconv1(x), encode1), dim=1) # (128, H, W)
        x = self.decoder1(x) # (64, H, W)
        x = self.output(x)

        return x

In [5]:
import torchsummary

model = UNet(num_classes=7).to(device)
torchsummary.summary(model, (3, 512, 512), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 512, 512]             224
       BatchNorm2d-2          [-1, 8, 512, 512]              16
              ReLU-3          [-1, 8, 512, 512]               0
            Conv2d-4          [-1, 8, 512, 512]             584
       BatchNorm2d-5          [-1, 8, 512, 512]              16
              ReLU-6          [-1, 8, 512, 512]               0
            Conv2d-7          [-1, 8, 512, 512]             584
       BatchNorm2d-8          [-1, 8, 512, 512]              16
              ReLU-9          [-1, 8, 512, 512]               0
        MaxPool2d-10          [-1, 8, 256, 256]               0
           Conv2d-11         [-1, 16, 256, 256]           1,168
      BatchNorm2d-12         [-1, 16, 256, 256]              32
             ReLU-13         [-1, 16, 256, 256]               0
           Conv2d-14         [-1, 16, 2

In [9]:
def mean_iou_score(pred, labels, num_classes=6):
    '''
    Compute mean IoU score over 6 classes
    '''
    mean_iou = []
    for i in range(num_classes):
        tp_fp = torch.sum(pred == i)
        tp_fn = torch.sum(labels == i)
        tp = torch.sum((pred == i) * (labels == i))
        # avoid nan
        if (tp_fp + tp_fn - tp) == 0:
            iou = 0.
        else:
            iou = tp / (tp_fp + tp_fn - tp)
        mean_iou.append(iou)

    return sum(mean_iou) / len(mean_iou)

#### Cross Entropy Loss

\begin{align}
L_{ce,i} \quad = \quad
\begin{cases}
  & -log(p_i)   \qquad, \text{ if } y_i= 1\\
  & -log(1-p_i) \; \;    ,\text{ if } y_i=0
\end{cases}
    \end{align}
#### Focal Loss

\begin{align}
L_{focal,i} \quad = \quad
\begin{cases}
  & -\alpha (1-p_i)^rlog(p_i)   \qquad     ,\text{ if } y_i= 1\\
  & -(1-\alpha)p_i^rlog(1-p_i)  \ \ \ \,   ,\text{ if } y_i=0
\end{cases}
    \end{align}

In [10]:
from torch import nn


class focal_loss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2): # Assume that the positive and negative sample ratio is 3:1
        super(focal_loss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.CrossEntropyLoss = nn.CrossEntropyLoss(ignore_index=6) # 用于指定在计算损失时应忽略的类别索引,参数被设置为背景类别的索引，以确保这些像素不会影响模型的训练

    def forward(self, preds, labels):
        Lce = -self.CrossEntropyLoss(preds, labels) # log(pi) or log(1-pi)
        Lfocal = -1 * (self.alpha) * ((1-torch.exp(Lce)) ** self.gamma) * Lce

        return Lfocal

In [11]:
from tqdm import tqdm


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch
    model.train() # to training mode.
    epoch_loss, epoch_iou = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True) # move data to GPU

        # Ensure the target labels are of type Long
        y = y.to(torch.long)
        # Compute prediction loss
        pred = model(x)
        loss = loss_fn(pred, y)
        # Optimization by gradients
        optimizer.zero_grad() # set prevision gradient to 0
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        pred = torch.argmax(pred, dim=1) # 沿着通道维度选择具有最高分数的通道
        pred = pred.to(torch.float)  # 将目标标签转换为浮点数类型
        # log
        epoch_loss += loss.item() # tensor -> python value
        epoch_iou += mean_iou_score(pred, y).item()

    # return avg loss of epoch, iou of epoch
    return epoch_loss/num_batches, epoch_iou/num_batches


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss = 0
    epoch_iou = 0
    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            y = y.to(torch.long)
            # Compute prediction loss
            pred = model(x)
            loss = loss_fn(pred, y)

            pred = torch.argmax(pred, dim=1) # 沿着通道维度选择具有最高分数的通道
            pred = pred.to(torch.float)  # 将目标标签转换为浮点数类型

            # write to logs
            epoch_loss += loss.item()
            epoch_iou += mean_iou_score(pred, y).item()

    return epoch_loss/num_batches, epoch_iou/num_batches

In [15]:
EPOCHS = 100
logs = {
    'train_loss': [], 'val_loss': [],
    'train_mean_iou': [], 'val_mean_iou': [],
}

model = UNet().to(device)
loss_fn = focal_loss() # classification for each pixel
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Earlystopping
patience = 5
counter = 0
best_IOU = 0

for epoch in tqdm(range(EPOCHS)):
    train_loss, train_mean_iou = train(train_loader, model, loss_fn, optimizer)
    val_loss, val_mean_iou = test(val_loader, model, loss_fn)
    logs['train_loss'].append(train_loss)
    logs['val_loss'].append(val_loss)
    logs['train_mean_iou'].append(train_mean_iou)
    logs['val_mean_iou'].append(val_mean_iou)

    print(f'EPOCH: {(epoch+1):04d} train_loss: {train_loss:.4f} val_loss: {val_loss:.4f}, train_mean_iou: {train_mean_iou:.3f}, val_mean_iou: {val_mean_iou:.3f}')

    # On epoch end
    torch.save(model.state_dict(), f'/content/drive/MyDrive/NTU_DLCV/p3_data/UNet_result/epoch{epoch+1}model.pth')
    # check improvement
    if best_IOU < val_mean_iou:
        counter = 0
        best_IOU = val_mean_iou
        torch.save(model.state_dict(), "/content/drive/MyDrive/NTU_DLCV/p3_data/UNet_result/BestIOU.pth")
        print('Best_IOU saved!')
    else:
        counter += 1
    if counter >= patience:
        print("Earlystop!")
        break

100%|██████████| 200/200 [00:41<00:00,  4.93it/s]
                                                 

EPOCH: 0001 train_loss: 0.2291 val_loss: 0.1552, train_mean_iou: 0.193, val_mean_iou: 0.143


  1%|          | 1/100 [01:40<2:46:29, 100.90s/it]

Best_IOU saved!



100%|██████████| 200/200 [00:41<00:00,  4.97it/s]
                                                 

...

EPOCH: 0030 train_loss: 0.0342 val_loss: 0.0428, train_mean_iou: 0.447, val_mean_iou: 0.201



 31%|███       | 31/100 [24:15<51:55, 45.15s/it]

EPOCH: 0031 train_loss: 0.0344 val_loss: 0.0437, train_mean_iou: 0.451, val_mean_iou: 0.185



 31%|███       | 31/100 [24:26<54:23, 47.30s/it]


FileNotFoundError: ignored